Google colab上でGymを走らせるには、xvfbやopenglなどをインストールしなければなりません。
####【おまじない】1度だけ実行すればいいです。

In [ ]:
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y

pyvirtualdisplayを使ってGymの環境を表示することができます。
####【おまじない】1度だけ実行すればいいです。

In [ ]:
!pip install pyvirtualdisplay
!pip install piglet

virtual displayをアクティブにするには下のスクリプトを一度だけ走らせる必要があります。【おまじない】

In [ ]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

#### Windowの人は実行します。
virtual displayを作るにはこれも走らせる必要があります。【おまじない】
#### Mac, Linuxの人は実行しません。

In [ ]:
# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

必要なモジュールをインポートします。

In [ ]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
#import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

show_video()とwrap_env(env)という関数を定義します。

In [ ]:
def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

ここからが自分で作る部分です。

## ランダムエージェント

In [ ]:
import gym

env = gym.make('CartPole-v1')
env = wrap_env(env)
observation = env.reset()
time = 0

while True:
    time += 1
    print("Step: {}".format(time))
    env.render()
    action = env.action_space.sample()
    observasion, reward, done, info = env.step(action)

    if done:
        break
    
env.close()
show_video()

In [ ]:
import gym

env = gym.make('CartPole-v1')
env = wrap_env(env)
steps = []

for episode in range(20):
    observation = env.reset()
    step = 0
    #for step in range(100):
    while True:
        env.render()
        action = env.action_space.sample()
        observasion, reward, done, info = env.step(action)

        if done:
            print("Episode {} finished after {} timesteps".format(episode + 1, step+1))
            break
        step += 1
    steps.append(step+1)
env.close()
show_video()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
plt.plot(steps)
plt.xlabel("Episode - 1")
plt.ylabel("Step")
plt.show()

## if-thenルールエージェント

In [ ]:
import gym
env = gym.make('CartPole-v1')
env = wrap_env(env)
steps = []
for episode in range(20):
    observation = env.reset()
    #for step in range(100):
    step = 0
    while True:
        env.render()
        #print(observation)
        _,_,th,_ = observation
        
        if th < 0:
            action = 0
        else:
            action = 1
        observation, reward, done, info = env.step(action)
        if done: 
            print("Episode {} finished after {} timesteps".format(episode + 1, step+1))
            break
        step += 1
    steps.append(step+1)
env.close()
show_video()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
plt.plot(steps)
plt.xlabel("Episode")
plt.ylabel("Step")
plt.show()

# Q学習エージェント

In [ ]:
import gym
import numpy as np
import random

class Agent:
    def __init__(self):
        self.Q = np.zeros((5**4,2))
        self.last_s = None
        self.last_a = None
    def quantize5(self, x, a, b):
        if x < -a:
            return 0
        elif x < -b:
            return 1
        elif x <= b:
            return 2
        elif x <= a:
            return 3
        else:
            return 4
    def quantize(self, obs):
        pos = self.quantize5(obs[0], 1.2, 0.2)
        vel = self.quantize5(obs[1], 1.5, 0.2)
        ang = self.quantize5(obs[2], 0.25, 0.02)
        acc = self.quantize5(obs[3], 1.0, 0.2)
        return pos + vel * 5 + ang * 25 + acc * 125
    def action(self, obs, episode, reward):
        s = self.quantize(obs)
        if random.random() > 0.5 * (1 / (episode + 1)):
            a = np.argmax(self.Q[s,:])
        else:
            a = random.randint(0,1)
        if self.last_s is not None:
            q = self.Q[self.last_s, self.last_a]
            q = q + 0.2 * (reward + 0.99 * np.max(self.Q[s,:]) - q)
            self.Q[self.last_s, self.last_a] = q
        self.last_s = s
        self.last_a = a
        return a

In [ ]:
agent = Agent()

env = gym.make('CartPole-v1')
env = wrap_env(env)
steps = []
for episode in range(100):
    observation = env.reset()
    reward = 0
    step = 0
    #done = 0
    #while True:
    for step in range(1000):
        env.render()
        action = agent.action(observation, episode, reward)
        observation, reward, done, info = env.step(action)
        if done:
            if step < 200:
                agent.action(observation, episode, -200)
            else:
                agent.action(observation, episode,  1)
            break

        step += 1
    print('Episode {} finished after {} timesteps'.format(episode+1, step+1))
    steps.append(step+1)
env.close()
show_video()


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure()
plt.plot(steps)
plt.xlabel("Episode")
plt.ylabel("Step")
plt.show()